# ByoTrack

## Loading a video 

In [ ]:
import matplotlib.pyplot as plt
import cv2

from byotrack.video.reader import VideoReader
from byotrack.video.transforms import VideoTransformConfig

In [ ]:
# Simply open a video
video = VideoReader.open("path/to/video")

In [ ]:
# A transform can be added to normalize and aggregate channels

transform_config = VideoTransformConfig(normalize=True, q_min=0.01, q_max=0.999)
video.set_transform(transform_config)

# Show the min max value used to clip and normalize
print(video.transform.normalizer.mini, video.transform.normalizer.maxi)

In [ ]:
# Display the first frame

plt.figure(figsize=(24, 16), dpi=100)
plt.imshow(video.retrieve())
plt.show()

In [ ]:
# Visualisation of the whole video with opencv

has_next = True
while has_next:
    frame, has_next = video.read()

    # Display the resulting frame
    cv2.imshow('Frame', frame)
    cv2.setWindowTitle('Frame', f'Frame {video.frame_id} / {video.length}')

    # Press Q on keyboard to  exit
    key = cv2.waitKey(video.fps if video.fps != -1 else 20) & 0xFF

    if key == ord('q'):
        break

    if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) <1:
        break


# Closes all the frames
cv2.destroyAllWindows()

## Detections on a video: Example of SpotDetector

In [ ]:
import numpy as np
import torch

from byotrack.detector.spot_detector import SpotDetector

In [ ]:
# Create the detector object with its hyper parameters

detector = SpotDetector(scale=1, k = 3.0, min_area=5, batch_size=50, device=torch.device("cpu"))

In [ ]:
# Run the detection process on the current video

detections_sequence = detector.run(video)

In [ ]:
# Display the detections with opencv
# Use w and x to move backward/forward in the video

i = 0
while True:
    mask = (detections_sequence[i].segmentation.numpy() != 0).astype(np.uint8) * 255
    
    # Display the resulting frame
    cv2.imshow('Frame', mask)
    cv2.setWindowTitle('Frame', f'Frame {i} / {len(detections_sequence)} - Number of detections: {detections_sequence[i].length}')

    # Press Q on keyboard to  exit
    key = cv2.waitKey(video.fps if video.fps != -1 else 20) & 0xFF

    if key == ord('q'):
        break

    if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) <1:
        break
        
    if key == ord("w"):
        i = (i - 1) % len(detections_sequence)

    if key == ord("x"):
        i = (i + 1) % len(detections_sequence)

cv2.destroyAllWindows()

In [ ]:
# Set hyperparameters manually on the video
# Use w/x to move backward/forward in the video
# Use c/v to update k (the main hyperparameter)
# You can restard with another scale/min_area

K_SPEED = 0.01

i = 0
video.seek(i)
frame = video.retrieve()

detector = SpotDetector(scale=1, k = 3.0, min_area=5.0, device=torch.device("cpu"))

while True:
    detections = detector.detect(frame[None, ...])[0]
    mask = (detections.segmentation.numpy() != 0).astype(np.uint8) * 255

    # Display the resulting frame
    cv2.imshow('Frame', mask)
    cv2.setWindowTitle('Frame', f'Frame {i} / {video.length} - k={detector.k} - Num detections: {detections.length}')

    # Press Q on keyboard to  exit
    key = cv2.waitKey() & 0xFF

    if key == ord('q'):
        break

    if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) <1:
        break
        
    if key == ord("w"):
        i = (i - 1) % video.length
        video.seek(i)
        frame = video.retrieve()

    if key == ord("x"):
        i = (i + 1) % video.length
        video.seek(i)
        frame = video.retrieve()
        
    if key == ord("c"):
        detector.k = detector.k * (1 - K_SPEED)

    if key == ord("v"):
        detector.k = detector.k * (1 + K_SPEED)
    

cv2.destroyAllWindows()

## Link detections: Example of IcyEMHTLinker

In [ ]:
from byotrack import Track
from byotrack.linker.icy_emht.icy_emht import IcyEMHTLinker

In [ ]:
# Create the linker object with icy path
linker = IcyEMHTLinker("path/to/icy/dir/")

In [ ]:
# You can set the expected motion of particles

linker.motion = linker.Motion.BROWNIAN  # Already by default

In [ ]:
# Run the linker given a video (Unused) and detections

tracks = linker.run(None, detections_sequence[:50])

In [ ]:
# Visualize tracks existence in time

# Transform into tensor
tracks_tensor = Track.tensorize(tracks)
print(tracks_tensor.shape)  # N_frame x N_track x D

mask = ~ torch.isnan(tracks_tensor).any(dim=2)

plt.figure(figsize=(24, 16), dpi=100)
plt.xlabel("Track id")
plt.ylabel("Frame")
plt.imshow(mask)
plt.show()

In [ ]:
# Visualize tracks with opencv
# Use w/x to move backward/forward in the video (or space to run the video)

fps = 20
running = False

video.seek(0)
true_frame = video.retrieve()

while True:
    if running:
        true_frame, has_next = video.read()

    frame = true_frame.copy()

    # Add tracklets
    for track in tracks:
        point = track[video.frame_id]
        if torch.isnan(point).any():
            continue
            
        x, y = point.round().to(torch.int).tolist()

        color = (0, 0, 255)  # Red

        cv2.circle(frame, (x, y), 5, color)
        cv2.putText(frame, str(track.identifier % 10), (x + 4, y - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.3, color)

    # Display the resulting frame
    cv2.imshow('Frame', frame)
    cv2.setWindowTitle('Frame', f'Frame {video.frame_id} / {video.length}')

    # Press Q on keyboard to  exit
    key = cv2.waitKey(int(1/fps * 1000)) & 0xFF

    if key == ord('q'):
        break

    if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) <1:
        break

    if key == ord(" "):
        running = not running

    if not running and key == ord("w"):  # Prev
        try:
            video.seek(video.frame_id - 4)
        except:
            pass

        true_frame = video.retrieve()

    if not running and key == ord("x"):  # Next
        true_frame, has_next = video.read()

# Closes all the frames
cv2.destroyAllWindows()

## End-to-end tracking: Example of MultiStepTracker

In [ ]:
from byotrack.tracker import MultiStepTracker

In [ ]:
# Create all the steps: Detector, Linker[, Refiner]
# Then the tracker

detector = SpotDetector(scale=1, k = 3.0, min_area=5, batch_size=50, device=torch.device("cpu"))
linker = IcyEMHTLinker("path/to/icy/dir")

tracker = MultiStepTracker(detector, linker)

In [ ]:
tracks = tracker.run(video)

In [ ]:
# Visualize tracks existence in time

# Transform into tensor
tracks_tensor = Track.tensorize(tracks)
print(tracks_tensor.shape)  # N_frame x N_track x D

mask = ~ torch.isnan(tracks_tensor).any(dim=2)

plt.figure(figsize=(24, 16), dpi=100)
plt.xlabel("Track id")
plt.ylabel("Frame")
plt.imshow(mask)
plt.show()

In [ ]:
# Visualize tracks with opencv
# Use w/x to move backward/forward in the video (or space to run the video)

fps = 20
running = False

video.seek(0)
true_frame = video.retrieve()

while True:
    if running:
        true_frame, has_next = video.read()

    frame = true_frame.copy()

    # Add tracklets
    for track in tracks:
        point = track[video.frame_id]
        if torch.isnan(point).any():
            continue
            
        x, y = point.round().to(torch.int).tolist()

        color = (0, 0, 255)  # Red

        cv2.circle(frame, (x, y), 5, color)
        cv2.putText(frame, str(track.identifier % 10), (x + 4, y - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.3, color)

    # Display the resulting frame
    cv2.imshow('Frame', frame)
    cv2.setWindowTitle('Frame', f'Frame {video.frame_id} / {video.length}')

    # Press Q on keyboard to  exit
    key = cv2.waitKey(int(1/fps * 1000)) & 0xFF

    if key == ord('q'):
        break

    if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) <1:
        break

    if key == ord(" "):
        running = not running

    if not running and key == ord("w"):  # Prev
        try:
            video.seek(video.frame_id - 4)
        except:
            pass

        true_frame = video.retrieve()

    if not running and key == ord("x"):  # Next
        true_frame, has_next = video.read()

# Closes all the frames
cv2.destroyAllWindows()